# VERSI 1 (tiap dokumen punya topik)

In [3]:
from tqdm import tqdm

In [103]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import nltk
from nltk.tokenize import RegexpTokenizer

In [8]:
df = pd.read_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/df_final.csv').head(10)

In [84]:
df = df.drop(columns = ['Unnamed: 0'])

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         10 non-null     int64 
 1   divisions          10 non-null     object
 2   title              10 non-null     object
 3   abstract           8 non-null      object
 4   subjects           10 non-null     object
 5   publication        6 non-null      object
 6   publisher          9 non-null      object
 7   creators           10 non-null     object
 8   date               10 non-null     object
 9   keywords           1 non-null      object
 10  uri                10 non-null     object
 11  new_abstract       8 non-null      object
 12  text_raw           10 non-null     object
 13  language           10 non-null     object
 14  language_accuracy  10 non-null     object
 15  fulltext           10 non-null     object
 16  fulltext_stopword  10 non-null     object
 17  

In [114]:
tokenizer = RegexpTokenizer(r'\w+')

topik = []

for index in range(len(df)):
  # Ambil dataset contoh (dalam kasus ini, dataset 20 newsgroups)
  teks = df['fulltext_stopword'].iloc[index]
  data = tokenizer.tokenize(teks)

  # Proses vektorisasi teks dengan TF-IDF
  tfidf_vectorizer = TfidfVectorizer(max_features=5000)
  tfidf_matrix = tfidf_vectorizer.fit_transform(data)

  # Terapkan LSA dengan TruncatedSVD
  num_topics = 4  # Jumlah topik yang ingin diidentifikasi
  lsa_model = TruncatedSVD(n_components=num_topics, random_state=42)
  lsa_topic_matrix = lsa_model.fit_transform(tfidf_matrix)

  # Ambil kata-kata kunci untuk setiap topik
  terms = tfidf_vectorizer.get_feature_names_out()
  top_keywords = 1  # Jumlah kata kunci untuk setiap topik (HANYA BISA DIAMBIL 1)

  kata_kunci = []

  for i, topic in enumerate(lsa_model.components_):
      top_keywords_idx = topic.argsort()[-top_keywords:][::-1]
      keywords = [terms[key_idx] for key_idx in top_keywords_idx]
      kata_kunci.append(keywords[0])

  topik.append(kata_kunci)

df['topik'] = topik

In [115]:
pd.DataFrame(df[['title','abstract','keywords','topik']])

,title,abstract,keywords,topik
0,TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...,NaN,"[ppm, sp, daphnia, metil]"
1,Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,NaN,"[pembelajaran, sejarah, siswa, media]"
2,Strategi Keberlanjutan Jasa Air Bersih Berbasi...,Persoalan penyediaan jasa air bersih merupakan...,NaN,"[air, bersih, masyarakat, dana]"
3,Metode Sexing Spermatozoa XY dan Aplikasi Inse...,NaN,NaN,"[kelamin, buatan, xy, metode]"
4,PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,Bawang merah merupakan komoditas pertanian pen...,"Bawang merah, Benih, Pelleting, TSS","[benih, tanah, bawang, merah]"
5,PENGENALAN BUNGA KRISAN POTONG DI DESA SUNGAI ...,Tujuan kegiatan ini adalah mengenalkan \r\nkom...,NaN,"[potong, komoditas, krisan, bunga]"
6,EFEKTIVITAS PRIMING PADA BENIH JEGUNG DALAM ME...,Kandungan Aluminium yang tinggi pada tanah Ult...,NaN,"[priming, benih, penelitian, perkecambahan]"
7,Pengaruh Literasi Digital Terhadap Etika Berme...,Tujuan dari penelitian ini adalah untuk menget...,NaN,"[penelitian, digital, literasi, etika]"
8,Perbaikan Korespondensi Produksi Cabai Merah (...,NaN,NaN,"[mt, diinduksi, produksi, merah]"
9,KEPENTINGAN INDONESIA MENGAKSESI KONVENSI APOS...,"Dalam era globalisasi, kerja sama antar negara...",NaN,"[publik, dokumen, negara, indonesia]"


# VERSI 2  (semua pencarian dijadikan 1 topik)

In [116]:
from tqdm import tqdm

In [117]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import nltk
from nltk.tokenize import RegexpTokenizer

In [118]:
df = pd.read_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/df_final.csv').head(10)

In [119]:
df = df.drop(columns = ['Unnamed: 0'])

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   divisions          10 non-null     object
 1   title              10 non-null     object
 2   abstract           8 non-null      object
 3   subjects           10 non-null     object
 4   publication        6 non-null      object
 5   publisher          9 non-null      object
 6   creators           10 non-null     object
 7   date               10 non-null     object
 8   keywords           1 non-null      object
 9   uri                10 non-null     object
 10  new_abstract       8 non-null      object
 11  text_raw           10 non-null     object
 12  language           10 non-null     object
 13  language_accuracy  10 non-null     object
 14  fulltext           10 non-null     object
 15  fulltext_stopword  10 non-null     object
 16  text_stemmed       10 non-null     object
dtype

In [131]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Ambil dataset contoh (dalam kasus ini, dataset 20 newsgroups)
data = df['fulltext_stopword']

# Proses vektorisasi teks dengan TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(data)

# Terapkan LSA dengan TruncatedSVD
num_topics = 5  # Jumlah topik yang ingin diidentifikasi (JUMLAH RELATIF DENGAN BANYAKNYA DATA)
lsa_model = TruncatedSVD(n_components=num_topics, random_state=42)
lsa_topic_matrix = lsa_model.fit_transform(tfidf_matrix)

# Ambil kata-kata kunci untuk setiap topik
terms = tfidf_vectorizer.get_feature_names_out()
top_keywords = 1  # Jumlah kata kunci untuk setiap topik

kata_kunci = []
topik = []

for i, topic in enumerate(lsa_model.components_):
    top_keywords_idx = topic.argsort()[-top_keywords:][::-1]
    keywords = [terms[key_idx] for key_idx in top_keywords_idx]
    kata_kunci.append(keywords)

for i in range(len(df)):
  topik.append(kata_kunci)

df['topik'] = topik

In [132]:
pd.DataFrame(df[['title','abstract','keywords','topik']])

,title,abstract,keywords,topik
0,TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...,NaN,"[[benih], [pembelajaran], [xy], [air], [dokumen]]"
1,Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,NaN,"[[benih], [pembelajaran], [xy], [air], [dokumen]]"
2,Strategi Keberlanjutan Jasa Air Bersih Berbasi...,Persoalan penyediaan jasa air bersih merupakan...,NaN,"[[benih], [pembelajaran], [xy], [air], [dokumen]]"
3,Metode Sexing Spermatozoa XY dan Aplikasi Inse...,NaN,NaN,"[[benih], [pembelajaran], [xy], [air], [dokumen]]"
4,PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,Bawang merah merupakan komoditas pertanian pen...,"Bawang merah, Benih, Pelleting, TSS","[[benih], [pembelajaran], [xy], [air], [dokumen]]"
5,PENGENALAN BUNGA KRISAN POTONG DI DESA SUNGAI ...,Tujuan kegiatan ini adalah mengenalkan \r\nkom...,NaN,"[[benih], [pembelajaran], [xy], [air], [dokumen]]"
6,EFEKTIVITAS PRIMING PADA BENIH JEGUNG DALAM ME...,Kandungan Aluminium yang tinggi pada tanah Ult...,NaN,"[[benih], [pembelajaran], [xy], [air], [dokumen]]"
7,Pengaruh Literasi Digital Terhadap Etika Berme...,Tujuan dari penelitian ini adalah untuk menget...,NaN,"[[benih], [pembelajaran], [xy], [air], [dokumen]]"
8,Perbaikan Korespondensi Produksi Cabai Merah (...,NaN,NaN,"[[benih], [pembelajaran], [xy], [air], [dokumen]]"
9,KEPENTINGAN INDONESIA MENGAKSESI KONVENSI APOS...,"Dalam era globalisasi, kerja sama antar negara...",NaN,"[[benih], [pembelajaran], [xy], [air], [dokumen]]"
